In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
kgs.cache_dir_read = kgs.cache_dir_write

/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
kgs.disable_caching = True

styleB not active yet
False False
(4901, 4901)


In [3]:
data = kgs.load_all_train_data(validation_only=True)[::25]
model.models[0].batch_size = 1
model.models[1].run_in_parallel = True
data_out = model.infer(data)
model.models[1].run_in_parallel = False

100%|██████████| 201/201 [00:20<00:00,  9.89it/s]


Clearing pytorch


Processing in parallel:   0%|          | 0/200 [00:00<?, ?it/s]

CUDA_VISIBLE_DEVICES= 0
/seismic/models/brendan/*.pth
Loading:  /seismic/models/brendan/unet2d_caformer_seed3_epochbest.pt
n_models: 1


Processing in parallel:  52%|█████▏    | 104/200 [01:02<00:21,  4.53it/s]

...

Processing in parallel:  89%|████████▉ | 178/200 [1:37:30<1:30:00, 245.47s/it]

True True
(4901, 1073)


Processing in parallel:  90%|████████▉ | 179/200 [1:41:42<1:26:35, 247.41s/it]

True True
(4901, 1073)


Processing in parallel: 100%|██████████| 200/200 [1:45:48<00:00, 31.74s/it]   


In [4]:
model.models[1].run_in_parallel = False
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    model.infer(this_data_out[:2])
    res['time_taken'].append((time.time()-t)/len(this_data_out[:2])/60)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

Inferring:   0%|          | 0/2 [00:00<?, ?it/s]

capturing graph
capturing graph


Inferring:   0%|          | 0/2 [00:00<?, ?it/s]

False True
(4901, 1073)


Inferring:  50%|█████     | 1/2 [04:07<04:07, 247.37s/it]

True True
(4901, 1073)


Inferring: 100%|██████████| 2/2 [00:00<00:00, 266.75it/s]




Brendan local score:  25.645244915397797
My local score:  21.792457149583463
Estimate public LB:  24.473260753737996


,family,score_brendan,score_mine,score_mine_max,time_taken
0,CurveFault_A,3.511432,3.511432,6.849573,0.004398
1,CurveFault_B,78.825357,78.825357,151.932449,0.004152
2,CurveVel_A,8.347980,8.347980,15.490612,0.004118
3,CurveVel_B,41.784173,41.784173,174.840204,0.004164
4,FlatFault_A,2.973499,2.973499,20.279864,0.004116
5,FlatFault_B,28.445296,28.445296,105.278163,0.004157
6,FlatVel_A,1.072704,0.038571,0.228571,0.594192
7,FlatVel_B,8.323500,0.022143,0.200000,0.456133
8,Style_A,33.723517,4.531130,11.191457,4.086111
9,Style_B,49.444991,49.444991,68.927527,0.004454


In [11]:
65000*4/60

4333.333333333333

In [9]:
65000*4/60*0.1

433.3333333333333